In [1]:
import pandas as pd
import numpy as np
import GPy
from collections import defaultdict

import seaborn as sns

%matplotlib inline

In [2]:
cd ../../../data/helpers/skills

/Users/Logan/Google Drive/Oxford/DPhil/future_employment/data/helpers/skills


In [3]:
X = pd.read_csv('skills_2009.csv')
Y = pd.read_csv('automation_targets.csv')
codes = pd.read_csv('codes_index.csv')
y = pd.read_csv('automation_y.csv')

In [4]:
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA

## UNSOLVED
# Using time series data – multiple years

## NOTES ABOUT GP MODELS
# optimize hyperparameters
# perform random restarts
# allow for multiple kernels
# optimize kernels
# optimize variances

## TO DO
# percentile regression model evaluation

# CREATE REGRESSION MODELS
from GPy.models import GPRegression
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

from GPy.models import GPClassification
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB

In [5]:
reg_models = [('gpreg', GPRegression),
			  ('bayes_ridge', BayesianRidge()),
			  ('Gboost', GradientBoostingRegressor()),
			  ('support_vec_reg', SVR())
			 ]

# CREATE CLASSIFICATION MODELS
class_models = [('gpclass', GPClassification),
				('ridge_class', RidgeClassifier()),
				('gbclass', GradientBoostingClassifier()),
				('support_vec_class', SVC()),
				('naivebayes', BernoulliNB())
		 		]

In [6]:
# CREATE INPUTS LIST
def get_array_percentiles(array):
		def percentile(x, array):
			return 100*np.mean(array <= x)

		return np.array(map(lambda x: percentile(x, array), array))

def get_percentiles(df):
	if isinstance(df, pd.DataFrame):
		new_df = df.copy()
		return new_df.apply(lambda x: get_array_percentiles(x), axis = 0)
	elif isinstance(df, np.ndarray):
		return np.apply_along_axis(get_percentiles, 0, a)
	else:
		print "TYPE ERROR; PLEASE INPUT pd.DataFrame OR np.ndarray"
		raise

In [7]:
X = pd.read_csv("skills_2009.csv")
X_skills, X_task, X_context = X, X, X
# X_skills = pd.read_csv("X_skills.csv")
# X_task = pd.read_csv("X_task.csv")
# X_context = pd.read_csv("X_context.csv")

In [8]:
Y = pd.read_csv("automation_targets.csv")
Y.columns = ["O*NET-SOC Code", "auto_15", "auto_9", "delta", "auto_delta_pct", "title"]

In [9]:
X = pd.read_csv("skills_2009.csv")
X_skills, X_task, X_context = X, X, X
codes = pd.read_csv("codes_index.csv")
full_X = pd.concat((X_skills, codes), axis = 1)
full_X = full_X.merge(Y, on = "O*NET-SOC Code")

In [11]:
inputs = []
for dX in [full_X]:
    pctiles = get_percentiles(dX)
    d_list =  [('dx', dX), ('pctile', pctiles)]
    for d in d_list:
        prefix = d[0]
        data = d[1]
        inputs.append((prefix, data.iloc[:,:-6]))
        inputs.append((prefix + '_greater_than', data[data.auto_delta_pct > 0].iloc[:,:-6]))
        inputs.append((prefix + '_less_than', data[data.auto_delta_pct < 0].iloc[:,:-6]))
# 		inputs.append(data[emp_delta_pct > 0])
# 		inputs.append(data[emp_delta_pct < 0])

# new_inputs = []
# for inp in inputs:
#     new_inputs.append(inp[0], inp[1].iloc[:,:-6])

# inputs = new_inputs

In [12]:
# FINAL DATA
y_out = full_X[['auto_9', 'auto_15', 'delta', 'auto_delta_pct']]
codes = full_X["O*NET-SOC Code"]

full_X = full_X.drop(['O*NET-SOC Code', 'auto_9', 'auto_15', 'delta', 'auto_delta_pct', 'title'], axis = 1)

In [13]:
# automation = pd.read_csv("y_automation.csv")
# computerisation = pd.read_csv("y_computerisation.csv")

In [14]:
# CREATE REG TARGETS
y_before = ('before', y_out.auto_9)
y_after = ('after', y_out.auto_15)
y_delta = ('delta', y_out.delta)
y_pct = ('delta_pct', y_out.auto_delta_pct)
reg_targets = [y_before, y_after, y_delta, y_pct]

new_reg_targets = []
for name, series in reg_targets:
    new_reg_targets.append((name, series))
    new_name = name + '_pctile'
    new_series = get_array_percentiles(series)
    new_reg_targets.append((new_name, new_series))
    
reg_targets = new_reg_targets

In [16]:
# CREATE CLASS TARGETS
class_targets = []
pos_threshold_values = [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00]
neg_threshold_values = map(lambda x: -x, pos_threshold_values)
threshold_values = pos_threshold_values + neg_threshold_values

for name, target in reg_targets:
    for sd_threshold_value in threshold_values:
        low_threshold = target > (np.mean(target) - sd_threshold_value*np.std(target))
        new_name = str(sd_threshold_value)+"_gt_" + name
        class_targets.append((new_name,low_threshold))

# 	mean_threshold = target > np.mean(target)
# 	new_name = 'mean_gt_' + name
# 	class_targets.append((name, mean_threshold))

In [18]:
X_inputs = inputs
y_inputs_reg, y_inputs_class = reg_targets, class_targets
models_reg, models_class = reg_models, class_models

In [92]:
X_inputs[1][1].index

Int64Index([  0,   2,   3,  15,  16,  24,  26,  29,  32,  39,
            ...
            719, 722, 727, 730, 736, 748, 749, 750, 752, 756],
           dtype='int64', length=225)

In [71]:
X = X_inputs[0]
y = y_inputs_reg[6]

In [79]:
y_name, y_targ = y[0], y[1] > 0
y = (y_name, y_targ)

In [87]:
tr_x, te_x, tr_y, te_y = train_test_split(X[1], y_targ)
m = SVC(probability = True)
m.fit(tr_x, tr_y)
preds = m.predict_proba(te_x)

In [88]:
preds

array([[ 0.71000304,  0.28999696],
       [ 0.71000631,  0.28999369],
       [ 0.71000708,  0.28999292],
       [ 0.71000471,  0.28999529],
       [ 0.71000694,  0.28999306],
       [ 0.71000643,  0.28999357],
       [ 0.71000517,  0.28999483],
       [ 0.71000517,  0.28999483],
       [ 0.71000609,  0.28999391],
       [ 0.71000438,  0.28999562],
       [ 0.71000517,  0.28999483],
       [ 0.71000514,  0.28999486],
       [ 0.71000605,  0.28999395],
       [ 0.7100016 ,  0.2899984 ],
       [ 0.71000665,  0.28999335],
       [ 0.71000234,  0.28999766],
       [ 0.71000521,  0.28999479],
       [ 0.71000702,  0.28999298],
       [ 0.71000432,  0.28999568],
       [ 0.71000318,  0.28999682],
       [ 0.71000598,  0.28999402],
       [ 0.71000561,  0.28999439],
       [ 0.71000616,  0.28999384],
       [ 0.71000653,  0.28999347],
       [ 0.71000644,  0.28999356],
       [ 0.71000641,  0.28999359],
       [ 0.71000507,  0.28999493],
       [ 0.71000593,  0.28999407],
       [ 0.71000362,

In [82]:
score_model(preds, te_y, X, ('SVC', m), 'classification')

ZeroDivisionError: float division by zero

In [32]:
#### aggregate models as list of tuples [(name, val), (name, val)]
# loop

def run_loop(X_inputs, y_inputs_reg = None, y_inputs_class = None, models_reg = None, models_class = None):
    ## returns all models in comparison, as dict
    tree = lambda: defaultdict(tree)
    evals = tree()
    
    # begin loop over inputs
    for X_inp in X_inputs: #FOR ALL INPUTS
        X_name, X = X_inputs[0], X_inputs[1]
        
        # check if null pred_type
        if y_inputs_reg and not y_inputs_class:
            pred_type = 'regression'
        
        elif y_inputs_class and not y_inputs_reg:
            pred_type = 'classification'
        
        elif not(y_inputs_reg or y_inputs_class):
            raise "Input a target variable!"
        
        else: # both are fulfilled; perform classification & regression
            new_dict = evals.copy()
            for pred_type in ['regression', 'classification']:
                if pred_type == 'regression':
                    y_inps = y_inputs_reg
                    mods = models_reg
                elif pred_type == 'classification':
                    y_inputs = y_inputs_class
                    mods = models_class
                else:
                    return "SOMETHING WRONG!"
                new_dict.update(run_model())
                evals = new_dict
        
        evals = run_model()
                
        def run_model(pred_type):
            model_evals = tree()
        
            # do inference over all targets

            for y_inp in y_inps:
                # format outcome data
                y_name, y = y_inp[0], y_inp[1]
                if y.ndim == 1:
                    y = y[:, np.newaxis]

                #format feature matrix data
                X_train, X_test, y_train, y_test = train_test_split(X, y)
                data = [X_train, X_test, y_train, y_test]
                new_data = []
                # check that all data is np.ndarry
                for d in data:
                    if not isinstance(data, np.ndarray):
                        new_data.append(np.array(data))
                    else:
                        new_data.append(data)
                X_train, X_test, y_train, y_test = new_data

                #loop over models
                for mod in mods:
                    model_name, model = models[0], models[1]
                    try:
                        model.fit(X_train, y_train)
                    except AttributeError:
                        model = model(X_train, y_train) # THESE ARE GPs!
                        model.optimize()
                        model.optimize_restarts(20)
                    y_pred = model.predict(X_test)
                    score = score_model(y_pred, y_test, X_inp, mod, score_type = pred_type) # CREATE TYPES
                    model_evals[model_name] = score
                return model_evals

        return evals


In [330]:
def score_model(y_pred, y_test, y_name, X_inputs, models, score_type):
    score = {}
    X_name, X = X_inputs[0], X_inputs[1]
    model_name, model = models[0], models[1]
    
    score['Model name'] = model_name
    score['Model'] = model
    score['X_name'] = X_name
    score['y_name'] = y_name
    score['y_pred'] = y_pred
    score['y_test'] = y_test

    if score_type == "classification":
        accuracy = -1
        precision = None
        recall = None
        specificity = None
        f1 = None
        ideal_cutoff = None

        chart_storage = np.array([None, None, None])
        for cutoff in np.arange(0, 1.001, 0.001):
            y_pred = y_pred > cutoff
            num_TP = float(np.sum((y_pred == 1) & (y_test == 1)))
            num_FP = float(np.sum((y_pred == 1) & (y_test == 0)))
            num_TN = float(np.sum((y_pred == 0) & (y_test == 0)))
            num_FN = float(np.sum((y_pred == 0) & (y_test == 1)))

            s_accuracy = np.mean(y_pred == y_test)
            s_precision = num_TP / (num_TP + num_FP)
            s_recall = num_TP / (num_TP + num_FN)
            s_specificity = num_TN / (num_TN + num_FP)
            s_f1 = (precision * recall) / (precision + recall)

            chart_storage.append([cutoff, s_recall, s_specificity])

            if s_accuracy > accuracy:
                ideal_cutoff = cutoff
                precision = s_precision
                recall = s_recall
                specificity = s_specificity 
                f1 = s_f1
                ideal_cutoff = s_ideal_cutoff

        def plot_chart(chart_storage):
            cutoffs = chart_storage[:,0]
            sensitivity = chart_storage[:,1]
            specificity = chart_storage[:,2]
            sns.plt.plot(sensitivity, specificity)
            sns.plt.close()

        eval_text = """
        Model 		| 	{}		|
        X name		|	{}		|
        Accuracy	|	{}		|
        Precision  	|	{}		|
        Recall 		|	{}		|
        Specificity |	{}		|
        F1			|	{}		|
        """.format(model_name,
                   X_name,
                   accuracy,
                   precision,
                   recalll,
                   specificity,
                   f1)
        score['Accuracy'] = accuracy
        score['Precision'] = precision
        score['Recall'] = recall
        score['Specificity'] = specificity
        score['F1'] = F1
        score['AUC_data'] = chart_storage
        score['chart'] = plot_chart # when called, call on plot_chart( score['AUC_data'])

        print eval_text
        return score

    elif score_type == 'regression':
        errors = y_test - y_pred
        se = float(np.std(errors)) / np.sqrt(len(errors))
        try:
            likelihood = model.likelihood
        except:
            likelihood = 'N/A'

        eval_text = """
        Model name  |     {}
        X name      |     {}
        Mean Y      |     {}
        Mean Y_hat  |     {}
        Mean error  |     {}
        SE          |     {}
        Likelihood  |     ##
        """.format(model_name,
                   X_name,
                   np.mean(y_test),
                   np.mean(y_pred),
                   np.mean(errors),
                   se
#                    likelihood
                   )
        score['likelihood'] = likelihood
        score['Standard Error'] = se
        score['mean y'] = np.mean(y_test)
        score['mean y_hat'] = np.mean(y_pred)
        score['mean_error'] = np.mean(errors)
        
        ## plot pca

        ## plot errors

        print eval_text
        return score
    else:
        return "Please input a valid score_type {'regression', 'classification'}"

In [ ]:
def do_ARD(model, feature_names):
	# uses ARD to find relevant features
	length_scales = model.length_scales # np.ndarray
	features = range(1, len(model.features) + 1)
	importances = 1./length_scales
	cutoff = 0.25 * min(importances) # note: arbitrary
	important_features = features * (importances >= cutoff)
	if_indices = np.trim_zeros(important_features)
	return zip(feature_names[if_indices], importances[if_indices])

In [99]:
t = X[1]

In [109]:
out = list(t.columns[:3])

In [114]:
a, b = t[out], t.drop(out, axis = 1)

In [127]:
auto = pd.read_csv('automation_targets.csv')

In [128]:
auto.columns

Index([u'O*NET-SOC Code', u'automation_15', u'automation_9', u'delta',
       u'delta_pct', u'Title'],
      dtype='object')

In [156]:
X = pd.read_table("../../databases/db09/Skills.txt")
X = X[X['Scale ID'] == 'LV']
X = X.pivot_table(values = 'Data Value', index = 'O*NET-SOC Code', columns = 'Element Name')
X = X.reset_index()

In [162]:
name = 'x'
output_columns = auto
subset_columns = 'delta_pct'
y_before = 'automation_9'
y_after = 'automation_15'

In [195]:
inps = generate_inputs(name, X, output_columns, subset_columns, y_before, y_after)

CREATING X's
TRANFORMING X's
CREATING y's


In [194]:
def generate_inputs(X_name, X_input, output_columns, subset_columns, y_before_name, y_after_name, job_codes = None):
    """
    X_name: name of X_input
    X_input: raw input with O*NET-SOC Code
    output_columns: interest variable df, like automation, with O*NET-SOC Code
    subset_columns: output columns you want to subset X on. Str or List
    y_before: name of column containing before target
    y_after: name of column containing after target
    job_codes: necessary only if 
    """
    X = X_input.copy()
    # first, format inputs
    if job_codes and "O*NET-SOC Code" not in X_input.columns:
        X = pd.concat((X_input, job_codes), axis = 1)

    X = X.merge(output_columns, on = 'O*NET-SOC Code', how = 'inner')


    # generate X subsets
    print "CREATING X's"
    if isinstance(subset_columns, str):
        subset_columns = [subset_columns]

    subset_inputs = []
    subset_inputs.append(['full', X])
    for subset in subset_columns:
        Xss = X[subset]
        s = subset
        subset_inputs.append(['pos_change_' + s, X[Xss > 0]])
        subset_inputs.append(['neg_change_' + s, X[Xss < 0]])
        subset_inputs.append(['gt_mean_' + s, X[Xss > Xss.mean()]])
        subset_inputs.append(['lt_mean_' + s, X[Xss < Xss.mean()]])

    # generate transforms
    print "TRANFORMING X's"
    transformed_inputs = []
    cols = list(output_columns.columns)
    for subset_input in subset_inputs:
        transformed_inputs.append(subset_input)
        name, df = subset_input[0], subset_input[1]
        retain = df[cols]
        of_interest = df.drop(cols, axis = 1)

        # percentile
        pct_df = get_percentiles(of_interest)
        pct_df = pd.concat((pct_df, retain), axis = 1)
        pct_name = 'pile_' + name

        # log change
        log_df = np.log(of_interest)
        log_df = pd.concat((log_df, retain), axis = 1)
        log_name = 'log_' + name

        transformed_inputs.append([pct_name, pct_df])
        transformed_inputs.append([log_name, log_df])

    final_inputs = []
    
    # generate y inputs
    print "CREATING y's"
    for X in transformed_inputs:
        new_y_inputs = []
        X_name = X[0]
        y_before, y_after = X[1][y_before_name], X[1][y_after_name]
        delta = y_after - y_before
        pct_delta = (y_after.astype(float) - y_before.astype(float)) / y_before
        pile_pct_delta = get_array_percentiles(pct_delta)
        pctile_before = get_array_percentiles(y_before)
        pctile_after = get_array_percentiles(y_after)

        new_y_inputs.append(['before', y_before])
        new_y_inputs.append(['after', y_after])
        new_y_inputs.append(['pct_delta', pct_delta])
        new_y_inputs.append(['pile_pct_delta', pile_pct_delta])
        new_y_inputs.append(['pctile_before', pctile_before])
        new_y_inputs.append(['pctile_after', pctile_after])

        final_y_inputs = []
        for name, y_input in new_y_inputs:
            final_y_inputs.append([name, y_input])
            final_y_inputs.append(['log_' + name, np.log(y_input)])

        X_final = X[1].drop(cols, axis = 1)
        for y_name, y_input_final in final_y_inputs:
            final_inputs.append([X_name, X_final, y_name, y_input_final])

    return final_inputs


def create_class_inputs(input_list):
    """
    Input list has to be structured as a list of lists, s.t. the inner list is constructed:

                [X_name, X, y_name, y]

    """
    class_list = []
    for i in input_list:
        X_name, X, y_name, y = i[0], i[1], i[2], i[3]

        # gt/lt than mean
        class_list.append([X_name, X, "gt_mean_" + y_name, y > np.mean(y)])
        class_list.append([X_name, X, "lt_mean_" + y_name, y < np.mean(y)])

        # gt/lt than 0
        if min(y) < 0 and max(y) > 0:
            class_list.append([X_name, X, "gt_0_" + y_name, y > 0])
            class_list.append([X_name, X, "lt_0_" + y_name, y < 0])

        # gt/lt Z std deviations from mean
        devs = np.array([0.5, 1, 1.5, 2, 2.5, 3])
        devs = np.hstack((devs, -devs))
        std = np.std(y)
        mean = np.mean(y)
        for sd in devs:
            new_y = y > (mean + sd*std)
            name = str(std) + "_gt_" + y_name
            class_list.append([X_name, X, name, new_y])

    return class_list

In [235]:
def create_class_inputs(input_list):
    """
    Input list has to be structured as a list of lists, s.t. the inner list is constructed:

                [X_name, X, y_name, y]

    """
    class_list = []
    for i in input_list:
        X_name, X, y_name, y = i[0], i[1], i[2], i[3]

        # gt/lt than mean
        class_list.append([X_name, X, "gt_mean_" + y_name, y > np.mean(y)])
        class_list.append([X_name, X, "lt_mean_" + y_name, y < np.mean(y)])

        # gt/lt than 0
        if min(y) < 0 and max(y) > 0:
            class_list.append([X_name, X, "gt_0_" + y_name, y > 0])
            class_list.append([X_name, X, "lt_0_" + y_name, y < 0])

        # gt/lt Z std deviations from mean
        devs = np.array([0.5, 1, 1.5, 2, 2.5, 3])
        devs = np.hstack((devs, -devs))
        std = np.std(y)
        mean = np.mean(y)
        for sd in devs:
            new_y = y > (mean + sd*std)
            name = str(std) + "_gt_" + y_name
            class_list.append([X_name, X, name, new_y])

    return class_list

# testing loop code

In [336]:

def run_loop(loop_type = 'regression', reg_inputs = None, class_inputs = None, reg_models = None, class_models = None):
    ## returns all models in comparison, as dict
    scores = defaultdict(list)
    
    if loop_type == "regression":
        inputs = reg_inputs
        models = reg_models
    else:
        inputs = class_inputs
        models = class_models

    for inp in inputs:
        X_name, X, y_name, y = inp

        # clean data in Numpy format
        if not isinstance(X, np.ndarray):
            X = np.array(X)
        if not isinstance(y, np.ndarray):
            y = np.array(y)
        if y.ndim == 1:
            y = y[:,np.newaxis]

        X_train, X_test, y_train, y_test = train_test_split(X, y)

        for model in models:
            model_name, mod = model
            try:
                if y_train.ndim > 1:
                    new_y_train = y_train.flatten()
                mod.fit(X_train, new_y_train)
                y_pred = mod.predict(X_test)
            except AttributeError:
                mod = mod(X_train, y_train)
                mod.optimize()
                mod.optimize_restarts(1)
                y_pred, y_pred_var = mod.predict(X_test)
                
            score = score_model(y_pred, y_test, y_name, (X_name, X), model, score_type = loop_type) # CREATE TYPES
            scores[model_name].append(score)
    return scores

In [331]:
first_inputs = inps[:2]
first_models = [('GPREG',GPRegression),('SVM',SVR())]

In [332]:
a, b, c, d = first_inputs[0]

In [333]:
tr_x, te_x, tr_y, te_y = train_test_split(np.array(b), np.array(d))